# 6. 파이썬 생태계와 통합하기

## 6.1. 시작하기
### 6.1.1. 파이썬 패키지 설치하기
```
pip install duckdb
```

In [1]:
import duckdb
duckdb.__version__

'1.2.1'

### 6.1.2. 데이터베이스 연결 열기

In [2]:
result = duckdb.sql('SELECT 42')
result.show()

┌───────┐
│  42   │
│ int32 │
├───────┤
│    42 │
└───────┘



In [3]:
result = duckdb.execute('SELECT 42')
row = result.fetchone()
print(row)

(42,)


## 6.2. 관계형 API 이용하기
### 6.2.1. 파이썬 API를 사용한 CSV 데이터 수집하기기

In [4]:
# 코드 6.1 CSV 파일 쿼리하기
con = duckdb.connect(database=':memory:')

con.execute("INSTALL httpfs")
con.execute("LOAD httpfs")

population = \
  con.read_csv("https://bit.ly/3KoiZR0")
type(population)

duckdb.duckdb.DuckDBPyRelation

In [5]:
con.execute("SELECT * from population limit 2").fetchall()

[('Afghanistan ',
  'ASIA (EX. NEAR EAST)         ',
  31056997,
  647500,
  '48,0',
  '0,00',
  '23,06',
  '163,07',
  700,
  '36,0',
  '3,2',
  '12,13',
  '0,22',
  '87,65',
  '1',
  '46,6',
  '20,34',
  '0,38',
  '0,24',
  '0,38'),
 ('Albania ',
  'EASTERN EUROPE                     ',
  3581655,
  28748,
  '124,6',
  '1,26',
  '-4,93',
  '21,52',
  4500,
  '86,5',
  '71,2',
  '21,09',
  '4,42',
  '74,49',
  '3',
  '15,11',
  '5,22',
  '0,232',
  '0,188',
  '0,579')]

In [6]:
(population
    .count("*")
    .show()
)

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          227 │
└──────────────┘



In [7]:
population.to_table("population")

In [8]:
population_table = con.table("population")

In [9]:
population_table.count("*").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          227 │
└──────────────┘



### 6.2.2. 쿼리 구성하기기

In [10]:
(population_table
  .filter('Population > 10000000')
  .project("Country, Population")
  .limit(5)
  .show()
)

┌──────────────┬────────────┐
│   Country    │ Population │
│   varchar    │   int64    │
├──────────────┼────────────┤
│ Afghanistan  │   31056997 │
│ Algeria      │   32930091 │
│ Angola       │   12127071 │
│ Argentina    │   39921833 │
│ Australia    │   20264082 │
└──────────────┴────────────┘



In [11]:
over_10m = population_table.filter('Population > 10000000')

In [12]:
(over_10m
  .aggregate("Region, CAST(avg(Population) AS int) as pop")
  .order("pop DESC")
)

┌─────────────────────────────────────┬───────────┐
│               Region                │    pop    │
│               varchar               │   int32   │
├─────────────────────────────────────┼───────────┤
│ ASIA (EX. NEAR EAST)                │ 192779730 │
│ NORTHERN AMERICA                    │ 165771574 │
│ LATIN AMER. & CARIB                 │  48643375 │
│ C.W. OF IND. STATES                 │  48487549 │
│ WESTERN EUROPE                      │  38955933 │
│ NORTHERN AFRICA                     │  38808343 │
│ NEAR EAST                           │  32910924 │
│ SUB-SAHARAN AFRICA                  │  30941436 │
│ EASTERN EUROPE                      │  23691959 │
│ OCEANIA                             │  20264082 │
├─────────────────────────────────────┴───────────┤
│ 10 rows                               2 columns │
└─────────────────────────────────────────────────┘

In [13]:
(over_10m
  .filter('"GDP ($ per capita)" > 10000')
  .count("*")
)

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│           20 │
└──────────────┘

In [14]:
(population_table
  .except_(over_10m)
  .aggregate("""
   Region,
   CAST(avg(population) AS int) AS population,
   count(*)
   """)
)

┌─────────────────────────────────────┬────────────┬──────────────┐
│               Region                │ population │ count_star() │
│               varchar               │   int32    │    int64     │
├─────────────────────────────────────┼────────────┼──────────────┤
│ OCEANIA                             │     643379 │           20 │
│ NEAR EAST                           │    2773978 │           11 │
│ C.W. OF IND. STATES                 │    5377686 │            7 │
│ SUB-SAHARAN AFRICA                  │    3322228 │           30 │
│ NORTHERN AMERICA                    │      43053 │            3 │
│ EASTERN EUROPE                      │    5426538 │            9 │
│ ASIA (EX. NEAR EAST)                │    2796374 │            9 │
│ LATIN AMER. & CARIB                 │    2154024 │           35 │
│ NORTHERN AFRICA                     │    3086881 │            2 │
│ BALTICS                             │    2394991 │            3 │
│ WESTERN EUROPE                      │    24071

In [15]:
eastern_europe = population_table \
  .filter("Region ~ '.*EASTERN EUROPE.*'")

In [16]:
(eastern_europe
  .intersect(over_10m)
  .project("Country, Population")
)

┌─────────────────┬────────────┐
│     Country     │ Population │
│     varchar     │   int64    │
├─────────────────┼────────────┤
│ Czech Republic  │   10235455 │
│ Poland          │   38536869 │
│ Romania         │   22303552 │
└─────────────────┴────────────┘

### 6.2.3.	SQL 쿼리하기

In [17]:
con.sql("""
SELECT count(*)
FROM over_10m
WHERE "GDP ($ per capita)" > 10000
""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│           20 │
└──────────────┘

In [18]:
con.execute("""
SELECT count(*)
FROM over_10m
WHERE "GDP ($ per capita)" > $gdp
""", {
  "gdp":10000
}).fetchone()


(20,)

## 6.3.	pandas 데이터프레임 쿼리하기
```pip install pandas```

In [19]:
import duckdb 
import pandas as pd

In [20]:
people = pd.DataFrame({
    "name": ["Michael Hunger", "Michael Simons", "Mark Needham"],
    "country": ["Germany", "Germany", "Great Britain"]
})

In [21]:
duckdb.sql("""
SELECT *
FROM people
WHERE country = 'Germany'
""")

┌────────────────┬─────────┐
│      name      │ country │
│    varchar     │ varchar │
├────────────────┼─────────┤
│ Michael Hunger │ Germany │
│ Michael Simons │ Germany │
└────────────────┴─────────┘

In [22]:
params = {"country": "Germany"}
duckdb.execute("""
SELECT *
FROM people
WHERE country <> $country
""", params).fetchdf()

,name,country
0,Mark Needham,Great Britain


In [23]:
(duckdb.sql("FROM people")
  .filter("country <> 'Germany'")
  .show()
)

┌──────────────┬───────────────┐
│     name     │    country    │
│   varchar    │    varchar    │
├──────────────┼───────────────┤
│ Mark Needham │ Great Britain │
└──────────────┴───────────────┘



## 6.4.	사용자 정의 함수

In [24]:
con.sql("""
select DISTINCT Region, length(Region) AS numChars
from population
""")

┌─────────────────────────────────────┬──────────┐
│               Region                │ numChars │
│               varchar               │  int64   │
├─────────────────────────────────────┼──────────┤
│ EASTERN EUROPE                      │       35 │
│ NORTHERN AFRICA                     │       35 │
│ C.W. OF IND. STATES                 │       20 │
│ NORTHERN AMERICA                    │       35 │
│ SUB-SAHARAN AFRICA                  │       35 │
│ NEAR EAST                           │       35 │
│ OCEANIA                             │       35 │
│ WESTERN EUROPE                      │       35 │
│ BALTICS                             │       35 │
│ ASIA (EX. NEAR EAST)                │       29 │
│ LATIN AMER. & CARIB                 │       23 │
├─────────────────────────────────────┴──────────┤
│ 11 rows                              2 columns │
└────────────────────────────────────────────────┘

In [25]:
def remove_spaces(field:str) -> str:
  if field:
    return field.lstrip().rstrip()
  else:
    return field

In [26]:
con.create_function('remove_spaces', remove_spaces)

In [27]:
con.sql("""
SELECT function_name, function_type, parameters, parameter_types, return_type
from duckdb_functions()
where function_name = 'remove_spaces'
""")

┌───────────────┬───────────────┬────────────┬─────────────────┬─────────────┐
│ function_name │ function_type │ parameters │ parameter_types │ return_type │
│    varchar    │    varchar    │ varchar[]  │    varchar[]    │   varchar   │
├───────────────┼───────────────┼────────────┼─────────────────┼─────────────┤
│ remove_spaces │ scalar        │ [col0]     │ [VARCHAR]       │ VARCHAR     │
└───────────────┴───────────────┴────────────┴─────────────────┴─────────────┘

In [28]:
con.sql("select length(remove_spaces(' foo '))")

┌────────────────────────────────┐
│ length(remove_spaces(' foo ')) │
│             int64              │
├────────────────────────────────┤
│                              3 │
└────────────────────────────────┘

In [29]:
con.remove_function('remove_spaces')

In [30]:
from duckdb.typing import *

con.create_function(
  'remove_spaces',
  remove_spaces,
  [(VARCHAR)],
  VARCHAR
)

In [33]:
con.sql("""
SELECT DISTINCT Region, length(Region) AS len1,
       remove_spaces(Region) AS cleanRegion,
       length(cleanRegion) AS len2
FROM population


""")

┌─────────────────────────────────────┬───────┬──────────────────────┬───────┐
│               Region                │ len1  │     cleanRegion      │ len2  │
│               varchar               │ int64 │       varchar        │ int64 │
├─────────────────────────────────────┼───────┼──────────────────────┼───────┤
│ SUB-SAHARAN AFRICA                  │    35 │ SUB-SAHARAN AFRICA   │    18 │
│ NEAR EAST                           │    35 │ NEAR EAST            │     9 │
│ BALTICS                             │    35 │ BALTICS              │     7 │
│ NORTHERN AMERICA                    │    35 │ NORTHERN AMERICA     │    16 │
│ EASTERN EUROPE                      │    35 │ EASTERN EUROPE       │    14 │
│ C.W. OF IND. STATES                 │    20 │ C.W. OF IND. STATES  │    19 │
│ OCEANIA                             │    35 │ OCEANIA              │     7 │
│ WESTERN EUROPE                      │    35 │ WESTERN EUROPE       │    14 │
│ LATIN AMER. & CARIB                 │    23 │ LATI

In [82]:
con.sql("""
UPDATE population
SET Region = remove_spaces(Region);
""")

In [83]:
con.sql("""
select DISTINCT Region, length(Region) AS numChars
from population
""")

┌──────────────────────┬──────────┐
│        Region        │ numChars │
│       varchar        │  int64   │
├──────────────────────┼──────────┤
│ NORTHERN AMERICA     │       16 │
│ C.W. OF IND. STATES  │       19 │
│ EASTERN EUROPE       │       14 │
│ NORTHERN AFRICA      │       15 │
│ NEAR EAST            │        9 │
│ OCEANIA              │        7 │
│ ASIA (EX. NEAR EAST) │       20 │
│ BALTICS              │        7 │
│ WESTERN EUROPE       │       14 │
│ LATIN AMER. & CARIB  │       19 │
│ SUB-SAHARAN AFRICA   │       18 │
├──────────────────────┴──────────┤
│ 11 rows               2 columns │
└─────────────────────────────────┘

In [84]:
from duckdb.typing import *
import locale

def convert_locale(field:str) -> float:
  locale.setlocale(locale.LC_ALL, 'de_DE')
  return locale.atof(field)

In [85]:
con.create_function('convert_locale', convert_locale)

In [86]:
con.sql("""
SELECT "Coastline (coast/area ratio)" AS coastline,
        convert_locale(coastline) as cleanCoastline,
        "Pop. Density (per sq. mi.)" as popDen,
        convert_locale(popDen) as cleanPopDen
FROM population
LIMIT 5
""")


┌───────────┬────────────────┬─────────┬─────────────┐
│ coastline │ cleanCoastline │ popDen  │ cleanPopDen │
│  varchar  │     double     │ varchar │   double    │
├───────────┼────────────────┼─────────┼─────────────┤
│ 0,00      │            0.0 │ 48,0    │        48.0 │
│ 1,26      │           1.26 │ 124,6   │       124.6 │
│ 0,04      │           0.04 │ 13,8    │        13.8 │
│ 58,29     │          58.29 │ 290,4   │       290.4 │
│ 0,00      │            0.0 │ 152,1   │       152.1 │
└───────────┴────────────────┴─────────┴─────────────┘

In [87]:
con.sql("""
ALTER TABLE population
ALTER "Coastline (coast/area ratio)"
SET DATA TYPE DOUBLE
USING
  convert_locale("Coastline (coast/area ratio)")
""")

## 6.5.	Apache Arrow와 Polars의 상호운용성
```pip install polars pyarrow```

In [88]:
import polars

population_table = con.table("population")

(population_table
   .limit(5)
   .pl()
  [["Country", "Region", "Population"]]
)

Country,Region,Population
str,str,i64
"""Afghanistan ""","""ASIA (EX. NEAR EAST)""",31056997
"""Albania ""","""EASTERN EUROPE""",3581655
"""Algeria ""","""NORTHERN AFRICA""",32930091
"""American Samoa ""","""OCEANIA""",57794
"""Andorra ""","""WESTERN EUROPE""",71201


In [89]:
arrow_table = population_table.to_arrow_table()

In [90]:
import pyarrow.compute as pc

(arrow_table
  .filter(pc.field("Region") == "NEAR EAST")
  .select(["Country", "Region", "Population"])
  .slice(length=5)
)


pyarrow.Table
Country: string
Region: string
Population: int64
----
Country: [["Bahrain ","Cyprus ","Gaza Strip ","Iraq ","Israel "]]
Region: [["NEAR EAST","NEAR EAST","NEAR EAST","NEAR EAST","NEAR EAST"]]
Population: [[698585,784301,1428757,26783383,6352117]]